# Deep learning-based automated rock classification via high-resolution drone-captured core sample imagery
***
### Domenico M. Crisafulli, Misael M. Morales, and Carlos Torres-Verdin
#### The University of Texas at Austin, 2024
***

## Build and Train NN-classifier
| Class             | OLD   | New   |
| ---               | ---   | ---   |
| Background        | 0     | 0     |
| Sandstone type 1  | 1     | 2     |
| Shaly Rock        | 2     | 3     |
| Sandstone type 2  | 3     | 4     |
| Carbonate         | 4     | 5     |
| Shale             | 5     | 6     |
| Sandstone type 3  | 6     | 7     |
| Box               | 10    | 1     |

In [2]:
import os
from time import time
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from tqdm import tqdm

from PIL import Image
import scipy.io as sio

import tensorflow as tf
import keras
from keras import Model
from keras import layers, losses, optimizers
from keras.activations import gelu
from keras.applications.resnet import ResNet50, preprocess_input

def check_tf_gpu():
    sys_info = tf.sysconfig.get_build_info()
    version, cuda, cudnn = tf.__version__, sys_info["cuda_version"], sys_info["cudnn_version"]
    count = len(tf.config.experimental.list_physical_devices())
    name  = [device.name for device in tf.config.experimental.list_physical_devices('GPU')]
    print('-'*60)
    print('----------------------- VERSION INFO -----------------------')
    print('TF version: {} | # Device(s) available: {}'.format(version, count))
    print('TF Built with CUDA? {} | CUDA: {} | cuDNN: {}'.format(tf.test.is_built_with_cuda(), cuda, cudnn))
    print(tf.config.list_physical_devices()[0],'\n', tf.config.list_physical_devices()[1])
    print('-'*60+'\n')
    return None

check_tf_gpu()

------------------------------------------------------------
----------------------- VERSION INFO -----------------------
TF version: 2.16.1 | # Device(s) available: 2
TF Built with CUDA? True | CUDA: 12.0 | cuDNN: 8
PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU') 
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
------------------------------------------------------------



In [3]:
def DeeplabV3Plus(image_size, num_classes):
    def convolution_block(block_input, num_filters=256, kernel_size=3, dilation_rate=1, use_bias=False):
        x = layers.Conv2D(num_filters, kernel_size=kernel_size, dilation_rate=dilation_rate, padding="same", 
                          use_bias=use_bias, kernel_initializer=keras.initializers.HeNormal())(block_input)
        x = layers.BatchNormalization()(x)
        x = layers.ReLU()(x)
        return x

    def DilatedSpatialPyramidPooling(dspp_input):
        dims = dspp_input.shape
        x = layers.AveragePooling2D(pool_size=(dims[-3], dims[-2]))(dspp_input)
        x = convolution_block(x, kernel_size=1, use_bias=True)
        out_pool = layers.UpSampling2D(
            size=(dims[-3] // x.shape[1], dims[-2] // x.shape[2]), interpolation="bilinear")(x)
        out_1 = convolution_block(dspp_input, kernel_size=1, dilation_rate=1)
        out_6 = convolution_block(dspp_input, kernel_size=3, dilation_rate=6)
        out_12 = convolution_block(dspp_input, kernel_size=3, dilation_rate=12)
        out_18 = convolution_block(dspp_input, kernel_size=3, dilation_rate=18)
        x = layers.Concatenate(axis=-1)([out_pool, out_1, out_6, out_12, out_18])
        output = convolution_block(x, kernel_size=1)
        return output

    model_input = keras.Input(shape=(image_size, image_size, 3))
    
    preprocessed = preprocess_input(model_input)
    resnet50 = ResNet50(weights="imagenet", include_top=False, input_tensor=preprocessed)

    x = resnet50.get_layer("conv4_block6_2_relu").output
    x = DilatedSpatialPyramidPooling(x)

    input_a = layers.UpSampling2D(
        size=(image_size // 4 // x.shape[1], image_size // 4 // x.shape[2]),
        interpolation="bilinear",
    )(x)
    input_b = resnet50.get_layer("conv2_block3_2_relu").output
    input_b = convolution_block(input_b, num_filters=48, kernel_size=1)

    x = layers.Concatenate(axis=-1)([input_a, input_b])
    x = convolution_block(x)
    x = convolution_block(x)
    x = layers.UpSampling2D(
        size=(image_size // x.shape[1], image_size // x.shape[2]),
        interpolation="bilinear",
    )(x)
    model_output = layers.Conv2D(num_classes, kernel_size=(1, 1), padding="same")(x)
    return keras.Model(inputs=model_input, outputs=model_output)

In [4]:
def patch_image(input_image, patch_h=252, patch_w=252, pad=4, channel_axis=-1):
    if channel_axis is not None:
        h, w, c = input_image.shape
    else:
        h, w = input_image.shape
    patches = []
    for i in range(0, h, patch_h):
        for j in range(0, w, patch_w):
            if channel_axis is not None:
                patch = input_image[i:i+patch_h, j:j+patch_w, :]
                patch = np.pad(patch, ((pad,pad),(pad,pad),(0,0)))
            else:
                patch = input_image[i:i+patch_h, j:j+patch_w]
                patch = np.pad(patch, ((pad,pad),(pad,pad)))
            patches.append(patch)
    return np.array(patches)

In [5]:
x_names_pictures = []
for root, dirs, files in os.walk('data'):
    for f in files:
        if (f.endswith('.jpg') or f.endswith('.JPG')) and f.startswith('DJI'):
            x_names_pictures.append(os.path.join(root, f))

y_names_masks = []
for root, dirs, files in os.walk('data'):
    for file in files:
        if file.endswith('.mat') and file.startswith('img_'):
            y_names_masks.append(os.path.join(root, file))

In [6]:
x_pictures = []
y_masks    = []
for i in tqdm(range(len(x_names_pictures)), desc='Loading images'):
    y = sio.loadmat(y_names_masks[i],simplify_cells=True)
    if 'AA' in y.keys():
        y_masks.append(y['AA'])
        x = np.array(Image.open(x_names_pictures[i]).convert('RGB'))
        x_pictures.append(x)

X_data = np.array(x_pictures)
y_data = np.array(y_masks)
print(X_data.shape, y_data.shape)

Loading images: 100%|██████████| 17/17 [00:18<00:00,  1.06s/it]


(15, 3024, 4032, 3) (15, 3024, 4032)


In [7]:
IMAGE_SIZE = 504
PADDING = 4
PADDED_IMAGE = IMAGE_SIZE+PADDING*2

x_data_patched = []
y_data_patched = []
for i in tqdm(range(X_data.shape[0]), desc='Patching images'):
    x = patch_image(X_data[i], patch_h=IMAGE_SIZE, patch_w=IMAGE_SIZE, pad=PADDING)
    x_data_patched.append(x)
    y = patch_image(y_data[i], patch_h=IMAGE_SIZE, patch_w=IMAGE_SIZE, pad=PADDING, channel_axis=None)
    y_data_patched.append(y)
    
x_data_patched = np.array(x_data_patched).reshape(-1, PADDED_IMAGE, PADDED_IMAGE, 3)
y_data_patched = np.array(y_data_patched).reshape(-1, PADDED_IMAGE, PADDED_IMAGE, 1)
print(x_data_patched.shape, y_data_patched.shape)

Patching images: 100%|██████████| 15/15 [00:00<00:00, 64.02it/s]


(720, 512, 512, 3) (720, 512, 512, 1)


In [8]:
idx = []
for i in tqdm(range(y_data_patched.shape[0]), desc='Nonzero Filter'):
    m = y_data_patched[i]
    p = np.sum(m)
    if p > 10000:
        idx.append(i)

x_data_nonzero = x_data_patched[idx]
y_data_nonzero = y_data_patched[idx]
print(x_data_nonzero.shape, y_data_nonzero.shape)

Nonzero Filter: 100%|██████████| 720/720 [00:00<00:00, 10669.14it/s]

(273, 512, 512, 3) (273, 512, 512, 1)


In [9]:
rand_idx = np.random.choice(x_data_nonzero.shape[0], size=x_data_nonzero.shape[0], replace=False)
n_train = 250

X_train = x_data_nonzero[rand_idx[:n_train]]/255
y_train = y_data_nonzero[rand_idx[:n_train]]

X_test = x_data_nonzero[rand_idx[n_train:]]/255
y_test = y_data_nonzero[rand_idx[n_train:]]

y_train[y_train==255] = 5
y_test[y_test==255] = 5

print('X - train: {} | test: {}'.format(X_train.shape, X_test.shape))
print('y - train: {} | test: {}'.format(y_train.shape, y_test.shape))
print('Labels - train: {} | test: {}'.format(np.unique(y_train), np.unique(y_test)))

X - train: (250, 512, 512, 3) | test: (23, 512, 512, 3)
y - train: (250, 512, 512, 1) | test: (23, 512, 512, 1)
Labels - train: [0 1 2 3 4 5 6] | test: [0 1 2 3 4 5]


In [ ]:
fig, axs = plt.subplots(2, 15, figsize=(25,4), sharex=True, sharey=True)
for j in range(15):
    ax1, ax2 = axs[0,j], axs[1,j]
    ax1.imshow(X_train[j])
    ax2.imshow(y_train[j])
plt.tight_layout()
plt.show()

In [10]:
model = DeeplabV3Plus(image_size=512, num_classes=6)
print('# parameters: {:,}'.format(model.count_params()))

optimizer = optimizers.Adam(learning_rate=1e-3)
criterion = losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=criterion, metrics=['accuracy', 'mse'])

start = time()
fit = model.fit(X_train, y_train,
                batch_size       = 8, 
                epochs           = 100, 
                validation_split = 0.2,
                shuffle          = True,
                verbose          = 1)
print('-'*30+'\n'+'Training time: {:.2f} minutes'.format((time()-start)/60))

2024-07-23 21:49:43.636467: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-23 21:49:43.636590: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-23 21:49:43.636624: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-23 21:49:43.840276: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-23 21:49:43.840355: I external/local_xla/xla/stream_executor

# parameters: 11,853,638
Epoch 1/100


I0000 00:00:1721789401.909377   83246 service.cc:145] XLA service 0x7fe560003d70 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1721789401.909429   83246 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-07-23 21:50:02.430955: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-23 21:50:04.135239: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907


In [ ]:
fig = plt.figure(figsize=(8,5))
ax1 = fig.add_subplot(111)
ax2 = ax1.twinx()
ax1.plot(fit.history['loss'], ls='--', label='train loss')
ax1.plot(fit.history['val_loss'], ls='--', label='val loss')
ax2.plot(fit.history['accuracy'], label='train accuracy', color='red')
ax2.plot(fit.history['val_accuracy'], label='val accuracy', color='green')
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Loss')
ax2.set_ylabel('Accuracy')
ax1.grid(True, which='both')
ax1.legend(loc='upper left')
ax2.legend(loc='lower right')
plt.show()

In [ ]:
y_train_pred = model.predict(X_train, verbose=0)
y_test_pred  = model.predict(X_test, verbose=0)
print('Pred - train: {} | test: {}'.format(y_train_pred.shape, y_test_pred.shape))

In [ ]:
fig, axs = plt.subplots(3, 15, figsize=(20,5), sharex=True, sharey=True)
for j in range(15):
    ax1, ax2, ax3 = axs[0,j], axs[1,j], axs[2,j]
    ax1.imshow(X_train[j])
    ax2.imshow(y_train[j])
    ax3.imshow(y_train_pred[j])
plt.tight_layout()
plt.show()

In [ ]:
X_data = np.load('data/x_images.npy')
y_data = np.load('data/y_images.npy')
print('X: {} | y: {}'.format(X_data.shape, y_data.shape))

idx = np.random.choice(range(len(X_data)), len(X_data), replace=False)
n_train = int(len(idx) * 0.8)

X_train, y_train = X_data[idx[:n_train]], y_data[idx[:n_train]]
X_test,  y_test  = X_data[idx[n_train:]], y_data[idx[n_train:]]
print('X - train: {} | test: {}'.format(X_train.shape, X_test.shape))
print('y - train: {} | test: {}'.format(y_train.shape, y_test.shape))

In [ ]:
def RockClassification(image_size:int=512, out_channels:int=1):
    
    def convolution_block(block_input, num_filters=256, kernel_size=3, dilation_rate=1, use_bias=False):
        x = layers.Conv2D(num_filters, kernel_size=kernel_size, dilation_rate=dilation_rate, padding="same", 
                          use_bias=use_bias, kernel_initializer=keras.initializers.HeNormal())(block_input)
        x = layers.BatchNormalization()(x)
        x = gelu(x)
        return x

    def DilatedSpatialPyramidPooling(dspp_input, num_filters=256):
        dims = dspp_input.shape
        x = layers.AveragePooling2D(pool_size=(dims[-3], dims[-2]))(dspp_input)
        x = convolution_block(x, num_filters, kernel_size=1, use_bias=True)
        out_pool = layers.UpSampling2D(
            size=(dims[-3] // x.shape[1], dims[-2] // x.shape[2]), interpolation="bilinear")(x)
        out_1 = convolution_block(dspp_input, num_filters, kernel_size=1, dilation_rate=1)
        out_6 = convolution_block(dspp_input, num_filters, kernel_size=3, dilation_rate=6)
        out_12 = convolution_block(dspp_input, num_filters, kernel_size=3, dilation_rate=12)
        out_18 = convolution_block(dspp_input, num_filters, kernel_size=3, dilation_rate=18)
        x = layers.Concatenate(axis=-1)([out_pool, out_1, out_6, out_12, out_18])
        output = convolution_block(x, num_filters, kernel_size=1)
        return output
    
    input = keras.Input(shape=(image_size, image_size, 1))
    _ = DilatedSpatialPyramidPooling(input, 256)
    _ = convolution_block(_, 128)
    _ = convolution_block(_, 64)
    _ = convolution_block(_, 32)
    _ = convolution_block(_, 16)
    output = layers.Conv2D(out_channels, kernel_size=(3,3), padding='same')(_)

    return Model(inputs=input, outputs=output)

***
## Train

In [ ]:
model = RockClassification()
print('# params: {:,}'.format(model.count_params()))
model.compile(optimizer=optimizers.Adam(1e-3), loss="binary_crossentropy", metrics=["accuracy"])

fit = model.fit(X_train, y_train,
                batch_size       = 4,
                epochs           = 10,
                validation_split = 0.2,
                shuffle          = True,
                verbose          = 1)

model.save_weights('rockClassification.weights.h5')
pd.DataFrame(fit.history).to_csv('fit_history.csv', index=False)

In [ ]:
model = DeeplabV3Plus(image_size=512, num_classes=1)
print('# params: {:,}'.format(model.count_params()))
model.compile(optimizer=optimizers.AdamW(1e-3, 4e-6), loss="binary_crossentropy", metrics=["accuracy"])

fit = model.fit(X_train, y_train,
                batch_size       = 8,
                epochs           = 10,
                validation_split = 0.2,
                shuffle          = True,
                verbose          = 1)

model.save_weights('DeeplabV3Plus.weights.h5')
pd.DataFrame(fit.history).to_csv('DeeplabV3Plus_fit_history.csv', index=False)

***

In [ ]:
model = RockClassification()
model.load_weights('rockClassification.weights.h5')
print('# params: {:,}'.format(model.count_params()))

losses = pd.read_csv('fit_history.csv')

plt.figure(figsize=(7,5))
plt.plot(losses.index, losses.accuracy, ls='-', marker='o', label='Accuracy')
plt.plot(losses.index, losses.val_accuracy, ls='--', marker='.', label='Val Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(facecolor='lightgrey', edgecolor='k')
plt.grid(True, which='both')
plt.tight_layout()
plt.show()

In [ ]:
model = DeeplabV3Plus(image_size=512, num_classes=1)
model.load_weights('DeeplabV3Plus.weights.h5')
print('# params: {:,}'.format(model.count_params()))

losses = pd.read_csv('DeeplabV3Plus_fit_history.csv')

plt.figure(figsize=(7,5))
plt.plot(losses.index, losses.accuracy, ls='-', marker='o', label='Accuracy')
plt.plot(losses.index, losses.val_accuracy, ls='--', marker='.', label='Val Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(facecolor='lightgrey', edgecolor='k')
plt.grid(True, which='both')
plt.tight_layout()
plt.show()

***
## Inference

In [ ]:
y_train_pred = model.predict(X_train[:6], verbose=0).round()
y_test_pred  = model.predict(X_test[:6], verbose=0).round()
print('Pred - train: {} | test: {}'.format(y_train_pred.shape, y_test_pred.shape))

In [ ]:
fig, axs = plt.subplots(3, 6, figsize=(15,5), sharex=True, sharey=True)
for j in range(6):
    ax1, ax2, ax3 = axs[0,j], axs[1,j], axs[2,j]
    im1 = ax1.imshow(X_train[j])
    im2 = ax2.imshow(y_train[j])
    im3 = ax3.imshow(y_train_pred[j])
    [a.set(xticks=[], yticks=[]) for a in [ax1, ax2, ax3]]
    [plt.colorbar(i, pad=0.04, fraction=0.046) for i in [im1,im2,im3]]
plt.tight_layout()
plt.show()

***
# END